# Toronto Bike Station ETL
We need to store information about Toronto bike stations in a table called toronto_bike_stations, the status of each bike station in the city of Toronto.
To do so, we will use the Open Data Portal as provider, and get the Bike Share Toronto data. The request is passed to the service using two separated URLs.

The URLS are:
* https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information
* https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status


The purpose is to answer these questions:

* How many bike stations are available in the city?
* What is the average bike availability?
* What are the 3 largest bike stations in the city?
* What are the 3 smallest bike stations in the city?

In [1]:
# import required libs
import json

import requests
import numpy as np
import pandas as pd
from pathlib import Path
from geopy.distance import distance
from sqlalchemy import create_engine


In [2]:
# Define some 'constants'
station_info_json =Path('data/input/station_info.json')
station_status_json = Path('data/input/station_status.json')

station_info_url = 'https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information'
station_status_url = 'https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status'

In [3]:
# request handler, gets the json payload and saves the data to a json file. 
# This is as not to make multiple calls for the same endpoint, for testing/expermintation purposes.
# and make sure we are catching the data at a specific time. 
def get_station_data(url, output_file):
    """
    Get request to the url endpoint and save the 
    data in the output_path.
    """
    response = requests.get(url)
    data = response.json()['data']['stations']
    with open(output_file, 'w') as outfile:
        json.dump(data, outfile)
        


## Extract Data from source

In [4]:
# Get the data of both endpoints and dump them into a json file.
get_station_data(station_info_url, station_info_json)
get_station_data(station_status_url, station_status_json)

In [5]:
# Transform the data to a tabulated form and explore each.
df_station_info = pd.read_json(station_info_json)
df_station_status = pd.read_json(station_status_json)

In [6]:
df_station_info.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,rental_methods,groups,obcn,nearby_distance,post_code,cross_street
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,0.0,Fort York Blvd / Capreol Ct,35,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],647-643-9607,500.0,NaN,NaN
1,7001,Lower Jarvis St / The Esplanade,REGULAR,43.647830,-79.370698,0.0,Lower Jarvis St / The Esplanade,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],416-617-9576,500.0,M5E 1R8,NaN
2,7002,St. George St / Bloor St W,REGULAR,43.667333,-79.399429,0.0,St. George St / Bloor St W,19,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],647-643-9615,500.0,NaN,NaN
3,7003,Madison Ave / Bloor St W,REGULAR,43.667158,-79.402761,NaN,Madison Ave / Bloor St W,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],647-631-4587,500.0,NaN,NaN
4,7004,University Ave / Elm St,REGULAR,43.656518,-79.389099,NaN,University Ave / Elm St,11,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[P7004-7047],647-643-9673,500.0,NaN,NaN


In [7]:
df_station_status.head()

,station_id,num_bikes_available,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,7000,13,"{'mechanical': 13, 'ebike': 0}",0,22,0,1,1,1,1603329536,False,IN_SERVICE
1,7001,3,"{'mechanical': 3, 'ebike': 0}",1,11,0,1,1,1,1603329337,False,IN_SERVICE
2,7002,3,"{'mechanical': 3, 'ebike': 0}",0,16,0,1,1,1,1603329468,False,IN_SERVICE
3,7003,0,"{'mechanical': 0, 'ebike': 0}",0,14,0,1,1,1,1603329365,False,IN_SERVICE
4,7004,1,"{'mechanical': 1, 'ebike': 0}",0,10,0,1,1,1,1603329459,False,IN_SERVICE


In [8]:
unique_id_info = len(df_station_info['station_id'].unique())
unique_id_status = len(df_station_status['station_id'].unique())
print(f'Info has {unique_id_info} unique ids')
print(f'status has {unique_id_status} unique ids')
# not quiet sure why info and status are not the same interms of id count. 

Info has 603 unique ids
status has 607 unique ids


In [9]:
# we will consider info to be the source and left join status on it.
# first assure no null ids in both
missing_id_info = np.sum(df_station_info['station_id'].isnull())
missing_id_status = np.sum(df_station_status['station_id'].isnull())
print(f'Missing ids in info: {missing_id_info}')
print(f'Missing ids in status: {missing_id_status}')

Missing ids in info: 0
Missing ids in status: 0


## Transform the dataframes and merge

In [10]:
# Left join status to info. Creating a new df instead of joining on df_station_info
df = pd.merge(df_station_info, df_station_status, how='left', on='station_id')
df.head()

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,rental_methods,groups,...,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,0.0,Fort York Blvd / Capreol Ct,35,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 13, 'ebike': 0}",0,22,0,1,1,1,1603329536,False,IN_SERVICE
1,7001,Lower Jarvis St / The Esplanade,REGULAR,43.647830,-79.370698,0.0,Lower Jarvis St / The Esplanade,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 3, 'ebike': 0}",1,11,0,1,1,1,1603329337,False,IN_SERVICE
2,7002,St. George St / Bloor St W,REGULAR,43.667333,-79.399429,0.0,St. George St / Bloor St W,19,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 3, 'ebike': 0}",0,16,0,1,1,1,1603329468,False,IN_SERVICE
3,7003,Madison Ave / Bloor St W,REGULAR,43.667158,-79.402761,NaN,Madison Ave / Bloor St W,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 0, 'ebike': 0}",0,14,0,1,1,1,1603329365,False,IN_SERVICE
4,7004,University Ave / Elm St,REGULAR,43.656518,-79.389099,NaN,University Ave / Elm St,11,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[P7004-7047],...,"{'mechanical': 1, 'ebike': 0}",0,10,0,1,1,1,1603329459,False,IN_SERVICE


In [11]:
# check missing values in the db before loading it to an sqlite db.
np.sum(df.isnull())

station_id                     0
name                           0
physical_configuration         0
lat                            0
lon                            0
altitude                      10
address                        0
capacity                       0
rental_methods                 0
groups                         0
obcn                           0
nearby_distance                0
post_code                    338
cross_street                 584
num_bikes_available            0
num_bikes_available_types      0
num_bikes_disabled             0
num_docks_available            0
num_docks_disabled             0
is_installed                   0
is_renting                     0
is_returning                   0
last_reported                  0
is_charging_station            0
status                         0
dtype: int64

In [15]:
# drop stations not in service
df[df['status'] == 'IN_SERVICE']

,station_id,name,physical_configuration,lat,lon,altitude,address,capacity,rental_methods,groups,...,num_bikes_available_types,num_bikes_disabled,num_docks_available,num_docks_disabled,is_installed,is_renting,is_returning,last_reported,is_charging_station,status
0,7000,Fort York Blvd / Capreol Ct,REGULAR,43.639832,-79.395954,0.0,Fort York Blvd / Capreol Ct,35,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 13, 'ebike': 0}",0,22,0,1,1,1,1603329536,False,IN_SERVICE
1,7001,Lower Jarvis St / The Esplanade,REGULAR,43.647830,-79.370698,0.0,Lower Jarvis St / The Esplanade,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 3, 'ebike': 0}",1,11,0,1,1,1,1603329337,False,IN_SERVICE
2,7002,St. George St / Bloor St W,REGULAR,43.667333,-79.399429,0.0,St. George St / Bloor St W,19,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 3, 'ebike': 0}",0,16,0,1,1,1,1603329468,False,IN_SERVICE
3,7003,Madison Ave / Bloor St W,REGULAR,43.667158,-79.402761,NaN,Madison Ave / Bloor St W,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 0, 'ebike': 0}",0,14,0,1,1,1,1603329365,False,IN_SERVICE
4,7004,University Ave / Elm St,REGULAR,43.656518,-79.389099,NaN,University Ave / Elm St,11,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[P7004-7047],...,"{'mechanical': 1, 'ebike': 0}",0,10,0,1,1,1,1603329459,False,IN_SERVICE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
598,7656,Bloor St W / Brock Ave,REGULAR,43.658988,-79.438715,0.0,Bloor St W / Brock Av,11,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 0, 'ebike': 0}",1,10,0,1,1,1,1603329301,False,IN_SERVICE
599,7657,1 Market St,ELECTRICBIKESTATION,43.646993,-79.370666,0.0,1 Market St,20,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 9, 'ebike': 0}",1,10,0,1,1,1,1603329393,True,IN_SERVICE
600,7658,Holland Park Ave / Oakwood Ave,REGULAR,43.686292,-79.438241,0.0,337 Oakwood Avenue,15,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 3, 'ebike': 0}",1,11,0,1,1,1,1603329489,False,IN_SERVICE
601,7659,Amroth Ave / Danforth Ave,REGULAR,43.685613,-79.311683,0.0,Amroth Ave / Danforth Ave,19,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",[],...,"{'mechanical': 5, 'ebike': 0}",0,14,0,1,1,1,1603329334,False,IN_SERVICE


In [12]:
#SQLite path
db_path = 'data/output/toronto_bike_stations'
engine = create_engine(f'sqlite:///{db_path}', echo=False)

In [201]:
#change object to string type before writing it to the sqlite db
for column in df.columns:
    if df[column].dtype == 'object':
        df[column] = df[column].apply(lambda x: str(x))

## Load the data
Load the data to an Sqlite file.

In [202]:
df.to_sql('stations', con=engine, if_exists='replace', index_label='station_id', index=False)


## Answer questions defined earlier
Will use the dataframe created earlier to answer the questions. 
Could load the data again using from sql or just execute raw SQL commands 
using the sqlalchemy engine. 

In [203]:
# first double check the data was loaded properly 
engine.execute("SELECT * FROM stations LIMIT 5;").fetchall()

[(7000, 'Fort York  Blvd / Capreol Ct', 'REGULAR', 43.639832, -79.395954, 0.0, 'Fort York  Blvd / Capreol Ct', 35, "['KEY', 'CREDITCARD', 'TRANSITCARD', 'PHONE']", '[]', '647-643-9607', 500.0, 'nan', 'nan', 13, "{'mechanical': 13, 'ebike': 0}", 0, 22, 0, 1, 1, 1, 1603324084, 0, 'IN_SERVICE'),
 (7001, 'Lower Jarvis St / The Esplanade', 'REGULAR', 43.64783, -79.370698, 0.0, 'Lower Jarvis St / The Esplanade', 15, "['KEY', 'CREDITCARD', 'TRANSITCARD', 'PHONE']", '[]', '416-617-9576', 500.0, 'M5E 1R8', 'nan', 7, "{'mechanical': 7, 'ebike': 0}", 1, 7, 0, 1, 1, 1, 1603324206, 0, 'IN_SERVICE'),
 (7002, 'St. George St / Bloor St W', 'REGULAR', 43.667333, -79.399429, 0.0, 'St. George St / Bloor St W', 19, "['KEY', 'CREDITCARD', 'TRANSITCARD', 'PHONE']", '[]', '647-643-9615', 500.0, 'nan', 'nan', 2, "{'mechanical': 2, 'ebike': 0}", 0, 17, 0, 1, 1, 1, 1603324077, 0, 'IN_SERVICE'),
 (7003, 'Madison Ave / Bloor St W', 'REGULAR', 43.667158, -79.402761, None, 'Madison Ave / Bloor St W', 15, "['KEY', '

In [204]:
# answering the questions. Alternating between SQL and pandas dataframe ¯\_(ツ)_/¯
num_stations = engine.execute("SELECT COUNT(*) as num_stations FROM stations;").fetchall()
avg_availability = df['num_bikes_available'].mean()
largest_stations = engine.execute("SELECT name FROM stations ORDER BY capacity DESC LIMIT 3;").fetchall()
smallest_stations = df[['name','capacity']].sort_values(by=['capacity'], ascending=True).head(3)['name']

In [205]:
# properly representing the answers function
def clean_engine_answers(values):
    result = []
    for value in values:
        result.append(value[0])
    return result

In [206]:
num_stations = clean_engine_answers(num_stations)
largest_stations = clean_engine_answers(largest_stations)

In [207]:
print('Number of stations:',num_stations)
print('Average bicycle availability:', avg_availability)
print('Largest stations:', largest_stations)
print('Smallest stations:', list(smallest_stations))

Number of stations: [603]
Average bicycle availability: 8.449419568822554
Largest stations: ['Bay St / Wellesley St W', 'Cherry Beach', 'Front St W / Yonge St (Hockey Hall of Fame)']
Smallest stations: ['Lisgar St / Dundas St SMART', 'Bloor St W / Pacific Ave', 'Chester Station (Green P)']


## Final Challenge
Find the closest station to this point:
lat = 43.661896,  Lon = -79.396160. The earth is a spheroid and lat, lon are in degrees so we need to calculate geographic distance.

In [208]:
df_lat_lon = df[['name','lat','lon']].copy()

In [209]:
def geodesic_distance(coord1, coord2):
    return distance(coord1, coord2).km
    
def find_closest_stations(df, current_coord):
    """
    Find the closest five stations to a given coordinate.
    Args:
        df(pd.DataFrame): dataframe which has stations name, 
            latitude and longitude.
        coordinates(tuple(float, float)): in the form (lat, lon)
    Return:
        five stations.
    
    """
    distances = []
    df['lat_diff'] = np.abs(df['lat'] - coords[0])
    df['lon_diff'] = np.abs(coords[1] - df['lat'])
    # Get the first 10 results to calculate distance on. The five are highly likely
    # in the first 10 sorted on lat and lon differences.
    # Idealy vectorize a distance calculaiton function.
    df = df.sort_values(by=['lat_diff','lon_diff'], ascending=True).head(10)
    df['coordinates'] = list(zip(df['lat'],df['lon']))
    for coord in df['coordinates']:
        distances.append(geodesic_distance(coord, current_coord))
    df['distances'] = distances
    return df.sort_values(by=['distances'], ascending=True)

In [210]:
coords= (43.661896, -79.396160)
find_closest_stations(df_lat_lon, coords)

,name,lat,lon,lat_diff,lon_diff,coordinates,distances
66,Willcocks St / St. George St,43.662085,-79.397735,0.000189,123.058245,"(43.662085, -79.397735)",0.128764
161,Spadina Ave / Willcocks St,43.661667,-79.401389,0.000229,123.057827,"(43.6616667, -79.40138888888889)",0.422533
254,Queen's Park Cres E / Grosvenor St - SMART,43.661803,-79.389682,0.000093,123.057963,"(43.661803, -79.389682)",0.522620
179,Central Tech (Harbord St),43.661975,-79.407896,0.000079,123.058135,"(43.661975, -79.407896)",0.946669
55,Jarvis St / Carlton St,43.662070,-79.376170,0.000174,123.058230,"(43.66207, -79.37617)",1.612512
146,Ossington Ave / Bloor St W,43.661705,-79.425734,0.000191,123.057865,"(43.661705, -79.425734)",2.385545
56,Parliament St / Gerrard St,43.662132,-79.365680,0.000236,123.058292,"(43.662132, -79.36567959999999)",2.458692
397,Concord Ave / Bloor St W - SMART,43.661890,-79.426790,0.000006,123.058050,"(43.66189, -79.42679)",2.470624
523,Oak St / Sumach St,43.661791,-79.361515,0.000105,123.057951,"(43.6617908, -79.361515)",2.794501
267,Queen St E / Pape Ave - SMART,43.661673,-79.337789,0.000223,123.057833,"(43.661673, -79.337789)",4.708294
